<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=250 ALIGN="right">
</figure>

## Analyse exam results for TU Delft course ASSESS

This notebook can be used to perform the analyses, provided you use EXACTLY the same excel template for analyzing the test 
results, as provided within the course. Please make sure you have matplotlib and pandas installed. 

*Developed by Dr. Ir. Hessel Winsemius (CEG - Water Management)*


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Below, a function is provided that reads metadata and the actual data into memory. The function also normalizes the data with respect to the maximum score per subquestion

In [ ]:
def examread(fn):
    """
    Read data from Excel tab called "Scores" into memory and return metadata, scores, and normalized scores
    Input: fn - string containing file name (excel)
    Output:
        los:    list of learning objectives 
        res:    the scores (with missing students excluded!)
        res_norm: scores normalised to the maximum score
    """
    # first read the grading info into memory, and only use the first rows for some metadata
    grade_info = pd.read_excel(fn, header=None, sheet_name='Scores', skiprows=9).iloc[:, 6:]
    numbers = grade_info.iloc[0:3, 0:1]
    no_part, no_qs, no_los = numbers.values[:, 0]

    qw = grade_info.iloc[7, 0:no_qs].values  # weight
    los = grade_info.iloc[8, 0:no_qs].values # learning objectives
    qn = grade_info.iloc[9, 0:no_qs].values # question names
    
    # now that we know the amount of questions and students, read the body of the results in memory
    body = pd.read_excel(fn, sheet_name='Scores', skiprows=18).iloc[:no_part]
    # extract the results only
    res = body.iloc[:, 6:]
    # remove no shows
    show_idx = res.sum(axis=1) > 1
    res = res[show_idx]
    sn = body.iloc[:, 3]
    name = body.iloc[:, 0]

    # normalize scores between zero and 100
    res_norm = pd.DataFrame(res/np.array(qw)*100, dtype=np.float64)
    # return any data we may need in the rest of the analysis
    return pd.Series(los, index=res.columns), res, res_norm   # ,  (no_part, no_qs, no_los), 

In [ ]:
# fill in the location of your results filename below 
# NOTE: use the exact Excel template as provided in the ASSESS training! 
# An example with random numbers is provided with this notebook, see "fn = ..." below.
fn = r'analysis_exam_EXAMPLE.xlsx'

# Below we read the data into memory (see function description to see what the variables mean)
los, res, res_norm = examread(fn)


We already have all data for a nice boxplot. Let's plot that!

In [ ]:
# sort columns on learning objectives
gr = los.index.groupby(los.values)
ordered_columns = [value for values in gr.values() for value in values]

plt.figure(figsize=(16, 8))
# res_norm.boxplot()
### comment line above, and uncomment line below to sort the questions according to learning objectives
res_norm[ordered_columns].boxplot()
los.name = 'los'
# los['los']
plt.xlabel('question number'); plt.ylabel('percentage correct [%]');
plt.ylim([-5, 105])
plt.title('Boxplot per subquestion')
plt.savefig('Boxplot.png', dpi=300, bbox_inches='tight')

Below we do some data analysis on the results (stored in DataFrame res)

In [ ]:
p = res_norm.mean()  # difficulty

max_score = res.max() # maximum score over all students
min_score = res.min() # minimum score over all students

# compute the item-total and item-rest correlation
sum_score = res.sum(axis=1)
sumrest_score = pd.DataFrame([res.drop(columns=i).sum(axis=1) for i in res.columns], index=res.columns).transpose()
Rit = res.corrwith(sum_score)
Rir = res.corrwith(sumrest_score)
stdev = res.std()



Let's make a good looking plot of the results

In [ ]:
f = plt.figure(figsize=(16, 10))

ax1 = plt.subplot(511)
Rit.plot(ax=ax1, color='b', label='Rit', xticks=range(len(Rit)))
Rir.plot(ax=ax1, color='r', label='Rir')
ax1.set_ylabel('Correlation coef. [-]')
ax1.set_xlim([0-0.5, len(Rit)-0.5])
ax1.set_xticklabels([])
ax1.legend()
ax1.grid()

ax2 = plt.subplot(512)
stdev.plot(ax=ax2, color='b', label='standard dev.', xticks=range(len(stdev)))
ax2.legend()
ax2.set_ylabel('Standard dev.')
ax2.set_xlim([0-0.5, len(stdev)-0.5])
ax2.set_xticklabels([])

ax2.grid()

ax3 = plt.subplot(513)
(res_norm.max()).plot(ax=ax3, color='b', label='maximum score', xticks=range(len(res_norm)))
(res_norm.min()).plot(ax=ax3, color='r', label='minimum score')
ax3.legend()
ax3.set_ylabel('Normalized score [%]')
ax3.set_xlim([0-0.5, len(stdev)-0.5])
ax3.set_xticklabels([])
ax3.grid()

ax4 = plt.subplot(514)
p.plot(ax=ax4, color='b', label='difficulty', xticks=range(len(p)))
ax4.set_ylabel('Difficulty [-]')
ax4.set_xlim([0-0.5, len(p)-0.5])
ax4.set_xticklabels([])

ax4.grid()

ax5 = plt.subplot(515)
res.mean().plot.bar(ax=ax5, color='k', label='mean score')
max_score.plot(ax=ax5, color='b', label='maximum score')
min_score.plot(ax=ax5, color='r', label='minimum score')
ax5.legend()
ax5.set_ylabel('Mean score')
ax5.grid()

f.savefig('Analysis_results.png', dpi=300, bbox_inches='tight')

Finally we compute summarised statistics for the entire exam

In [ ]:
N, N_q = res_norm.shape # nr of students, nr of questions
rest = res.sum(axis=1)

sigx = rest.std() # standard deviation of the total score
alpha = N_q/(N_q - 1)*(sigx**2-(stdev**2).sum())/sigx**2 # Cronbach's alpha

SEM_68 = sigx*(1-alpha)**.5 # 68% confidence interval of grades (SEM)

# make a nice plot
plt.figure()
ax = plt.axes()
rest.plot.hist(bins=range(5, 100, 10), ax=ax, xticks=range(0, 100, 10))
ax.set_xlabel('Grade [%]')
ax.set_ylabel('# Students')
ax.grid()
ax.set_title("Cronbach's alpha: {:.2f}, SEM: {:.2f}".format(alpha, SEM_68))
# TODO add results per learning objectives in fitting suplots
plt.savefig('summary_results.png', dpi=300, bbox_inches='tight')